In [1]:
#libraries
import acquire
import explore
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = acquire.get_telco_data()
df.head()

,payment_type_id,payment_type,internet_service_type_id,internet_service_type,contract_type_id,contract_type,customer_id,gender,senior_citizen,partner,...,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn
0,1,Electronic check,1,DSL,1,Month-to-month,0015-UOCOJ,Female,1,No,...,Yes,No,No,No,No,No,Yes,48.20,340.35,No
1,1,Electronic check,1,DSL,1,Month-to-month,0023-HGHWL,Male,1,No,...,No,No,No,No,No,No,Yes,25.10,25.1,Yes
2,1,Electronic check,1,DSL,1,Month-to-month,0067-DKWBL,Male,1,No,...,Yes,No,No,No,No,No,Yes,49.25,91.1,Yes
3,1,Electronic check,1,DSL,2,One year,0083-PIVIK,Male,0,No,...,Yes,Yes,Yes,Yes,Yes,No,No,81.25,5567.55,No
4,1,Electronic check,1,DSL,1,Month-to-month,0096-BXERS,Female,0,Yes,...,No,No,No,No,No,No,No,50.35,314.55,No


In [3]:
df.shape

(7043, 24)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   payment_type              7043 non-null   object 
 2   internet_service_type_id  7043 non-null   int64  
 3   internet_service_type     7043 non-null   object 
 4   contract_type_id          7043 non-null   int64  
 5   contract_type             7043 non-null   object 
 6   customer_id               7043 non-null   object 
 7   gender                    7043 non-null   object 
 8   senior_citizen            7043 non-null   int64  
 9   partner                   7043 non-null   object 
 10  dependents                7043 non-null   object 
 11  tenure                    7043 non-null   int64  
 12  phone_service             7043 non-null   object 
 13  multiple_lines            7043 non-null   object 
 14  online_s

### This will be in the prepare module
    _Check for and remove duplicates by customer_id. - None
    
    _Remove redundant columns: payment_type_id, internet_service_type_id, contract_type_id, & customer_id. - Completed
        -Used the following to drop redundant columns
        -df = df.drop(['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], axis = 1)
        -df.head(2)
    
    _Encode payment_type(Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)), internet_service_type(DSL, Fiber, None), contract_type(Month-to-month, One year, Two year), gender(male/female), partner(yes/no), multiple_line(yes/no), device_protection(yes/no), tech_support(yes/no), streaming_tv(yes/no), streaming_movies(yes/no), paperless_billing, & churn (yes/no).
    
    _Check for nulls and fill (Fill in total_charges' blanks with 0). - Completed
        -#find empty values. Tried isna, isnull, notna, notnull but did not shown any matches.
            -df.eq(' ').sum()
        -Used .replace to empty values with a 0 in total charges.
            -df = df.replace({'total_charges': ' '}, 0)
            -df.head()

    _Convert total_charges to float64. - Completed
        -#convert 'total_charges' to float and validate change.
        - df['total_charges'] = df['total_charges'].astype(float)
        - df.dtypes
    
    _Replace 'No phone service' & 'No internet service' with 'No'. - Completed
        -df.replace(to_replace = 'No internet service', value = 'No')
        -df.replace(to_replace = 'No phone service', value = 'No')

In [5]:
#Looking for unique values in "types"
df.contract_type.unique()

array(['Month-to-month', 'One year', 'Two year'], dtype=object)

In [6]:
#check to see if there are any duplicates by 'customer_id'
df.duplicated(subset = 'customer_id').unique()

array([False])

In [7]:
#find empty values. Tried isna, isnull, notna, notnull but did not shown any matches.
df.eq(' ').sum()

payment_type_id              0
payment_type                 0
internet_service_type_id     0
internet_service_type        0
contract_type_id             0
contract_type                0
customer_id                  0
gender                       0
senior_citizen               0
partner                      0
dependents                   0
tenure                       0
phone_service                0
multiple_lines               0
online_security              0
online_backup                0
device_protection            0
tech_support                 0
streaming_tv                 0
streaming_movies             0
paperless_billing            0
monthly_charges              0
total_charges               11
churn                        0
dtype: int64

In [10]:
#replaces empty values with a 0 in total charges.
df = df.replace({'total_charges': ' '}, 0)
df.head()

,payment_type_id,payment_type,internet_service_type_id,internet_service_type,contract_type_id,contract_type,customer_id,gender,senior_citizen,partner,...,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn
0,1,Electronic check,1,DSL,1,Month-to-month,0015-UOCOJ,Female,1,No,...,Yes,No,No,No,No,No,Yes,48.20,340.35,No
1,1,Electronic check,1,DSL,1,Month-to-month,0023-HGHWL,Male,1,No,...,No,No,No,No,No,No,Yes,25.10,25.1,Yes
2,1,Electronic check,1,DSL,1,Month-to-month,0067-DKWBL,Male,1,No,...,Yes,No,No,No,No,No,Yes,49.25,91.1,Yes
3,1,Electronic check,1,DSL,2,One year,0083-PIVIK,Male,0,No,...,Yes,Yes,Yes,Yes,Yes,No,No,81.25,5567.55,No
4,1,Electronic check,1,DSL,1,Month-to-month,0096-BXERS,Female,0,Yes,...,No,No,No,No,No,No,No,50.35,314.55,No


In [11]:
#verify empty values in total_charges have been replace.
df.eq(' ').sum()

payment_type_id             0
payment_type                0
internet_service_type_id    0
internet_service_type       0
contract_type_id            0
contract_type               0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
dtype: int64

In [12]:
df = df.drop(['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], axis = 1)
df.head(2)

,payment_type,internet_service_type,contract_type,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn
0,Electronic check,DSL,Month-to-month,Female,1,No,No,7,Yes,No,Yes,No,No,No,No,No,Yes,48.2,340.35,No
1,Electronic check,DSL,Month-to-month,Male,1,No,No,1,No,No phone service,No,No,No,No,No,No,Yes,25.1,25.1,Yes


In [13]:
df.shape

(7043, 20)

In [14]:
#convert 'total_charges' to float and validate change.
df['total_charges'] = df['total_charges'].astype(float)
df.dtypes

payment_type              object
internet_service_type     object
contract_type             object
gender                    object
senior_citizen             int64
partner                   object
dependents                object
tenure                     int64
phone_service             object
multiple_lines            object
online_security           object
online_backup             object
device_protection         object
tech_support              object
streaming_tv              object
streaming_movies          object
paperless_billing         object
monthly_charges          float64
total_charges            float64
churn                     object
dtype: object

In [15]:
df = df.replace(to_replace = 'No internet service', value = 'No')
df

,payment_type,internet_service_type,contract_type,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn
0,Electronic check,DSL,Month-to-month,Female,1,No,No,7,Yes,No,Yes,No,No,No,No,No,Yes,48.20,340.35,No
1,Electronic check,DSL,Month-to-month,Male,1,No,No,1,No,No phone service,No,No,No,No,No,No,Yes,25.10,25.10,Yes
2,Electronic check,DSL,Month-to-month,Male,1,No,No,2,Yes,No,Yes,No,No,No,No,No,Yes,49.25,91.10,Yes
3,Electronic check,DSL,One year,Male,0,No,No,64,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,81.25,5567.55,No
4,Electronic check,DSL,Month-to-month,Female,0,Yes,No,6,Yes,Yes,No,No,No,No,No,No,No,50.35,314.55,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Credit card (automatic),None,Month-to-month,Female,0,No,Yes,4,Yes,No,No,No,No,No,No,No,No,20.40,94.50,No
7039,Credit card (automatic),None,Two year,Male,0,Yes,Yes,28,Yes,No,No,No,No,No,No,No,Yes,20.30,487.95,No
7040,Credit card (automatic),None,Two year,Female,0,Yes,No,71,Yes,Yes,No,No,No,No,No,No,Yes,24.40,1725.40,No
7041,Credit card (automatic),None,Month-to-month,Female,0,No,No,6,Yes,No,No,No,No,No,No,No,No,19.70,129.55,No


In [16]:
df = df.replace(to_replace = 'No phone service', value = 'No')
df

,payment_type,internet_service_type,contract_type,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn
0,Electronic check,DSL,Month-to-month,Female,1,No,No,7,Yes,No,Yes,No,No,No,No,No,Yes,48.20,340.35,No
1,Electronic check,DSL,Month-to-month,Male,1,No,No,1,No,No,No,No,No,No,No,No,Yes,25.10,25.10,Yes
2,Electronic check,DSL,Month-to-month,Male,1,No,No,2,Yes,No,Yes,No,No,No,No,No,Yes,49.25,91.10,Yes
3,Electronic check,DSL,One year,Male,0,No,No,64,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,81.25,5567.55,No
4,Electronic check,DSL,Month-to-month,Female,0,Yes,No,6,Yes,Yes,No,No,No,No,No,No,No,50.35,314.55,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Credit card (automatic),None,Month-to-month,Female,0,No,Yes,4,Yes,No,No,No,No,No,No,No,No,20.40,94.50,No
7039,Credit card (automatic),None,Two year,Male,0,Yes,Yes,28,Yes,No,No,No,No,No,No,No,Yes,20.30,487.95,No
7040,Credit card (automatic),None,Two year,Female,0,Yes,No,71,Yes,Yes,No,No,No,No,No,No,Yes,24.40,1725.40,No
7041,Credit card (automatic),None,Month-to-month,Female,0,No,No,6,Yes,No,No,No,No,No,No,No,No,19.70,129.55,No


# Encode

In [17]:
#get_dummies creates a seperate df of booleans for the identified columns below. Cleaning for the decission tree.
dummy_df = pd.get_dummies(df[['dependents','phone_service','online_security','online_backup','payment_type','internet_service_type','contract_type','gender','partner','multiple_lines','device_protection','tech_support','streaming_tv','streaming_movies','paperless_billing','churn']], dummy_na=False, drop_first=[True, True])

In [18]:
#now drop the above two columns...
df = df.drop(columns=['dependents','phone_service','online_security','online_backup','payment_type','internet_service_type','contract_type','gender','partner','multiple_lines','device_protection','tech_support','streaming_tv','streaming_movies','paperless_billing','churn'])
#...and concatanate the dummies df with the prep's df.
df = pd.concat([df, dummy_df], axis=1)
df.head()

,senior_citizen,tenure,monthly_charges,total_charges,dependents_Yes,phone_service_Yes,online_security_Yes,online_backup_Yes,payment_type_Credit card (automatic),payment_type_Electronic check,...,contract_type_Two year,gender_Male,partner_Yes,multiple_lines_Yes,device_protection_Yes,tech_support_Yes,streaming_tv_Yes,streaming_movies_Yes,paperless_billing_Yes,churn_Yes
0,1,7,48.20,340.35,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,1,1,25.10,25.10,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,1
2,1,2,49.25,91.10,0,1,1,0,0,1,...,0,1,0,0,0,0,0,0,1,1
3,0,64,81.25,5567.55,0,1,1,1,0,1,...,0,1,0,1,1,1,1,0,0,0
4,0,6,50.35,314.55,0,1,0,0,0,1,...,0,0,1,1,0,0,0,0,0,0


# Split your data into train, validate, and test samples.